In [18]:
import json

# Read json within keys directory to pick up the keys
with open('/Users/vilourenco/Documents/GitHub/azure-ai-engineer/keys/keys.json') as f:
    keys = json.load(f)

In [19]:
import azure.cognitiveservices.speech as speech_sdk

# Initialize the SpeechConfig with the API key and region
speech_config = speech_sdk.SpeechConfig(keys["azure_foundry"]["api_key"], 'eastus')